In [0]:
dbutils.secrets.list(scope="fomular1-scope")

In [0]:
fomular1_key = dbutils.secrets.get(scope="fomular1-scope", key="formuladl1-key-account")

In [0]:
spark.conf.set("fs.azure.account.key.fomular1dl1.dfs.core.windows.net", fomular1_key)

In [0]:
circuits_df = spark.read.parquet("abfss://processed@fomular1dl1.dfs.core.windows.net/circuits").withColumnRenamed("location", "circuit_location")

In [0]:
race_df = spark.read.parquet("abfss://processed@fomular1dl1.dfs.core.windows.net/races").withColumnRenamed("name","race_name").withColumnRenamed("race_timestamp","race_date")

In [0]:
constructor_df = spark.read.parquet("abfss://processed@fomular1dl1.dfs.core.windows.net/constructors").withColumnRenamed("name","team")

In [0]:
drive_df = spark.read.parquet("abfss://processed@fomular1dl1.dfs.core.windows.net/drivers").withColumnRenamed("number","driver_number").withColumnRenamed("name","driver_name").withColumnRenamed("nationality","driver_nationality")

In [0]:
result_df = spark.read.parquet("abfss://processed@fomular1dl1.dfs.core.windows.net/results").withColumnRenamed("time","race_time")



In [0]:
race_circuit_df = race_df.join(circuits_df, race_df.circuit_id == circuits_df.circuit_id, "inner")\
.select(race_df.race_id, race_df.race_year, race_df.race_date, race_df.race_name, circuits_df.circuit_location)

In [0]:
race_result_df = race_circuit_df.join(result_df, race_circuit_df.race_id == result_df.race_id)\
.join(drive_df, result_df.driver_id == drive_df.driver_id)\
.join(constructor_df, result_df.constructor_id == constructor_df.constructor_id)


In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
final_df = race_result_df.select( "race_year", "race_date", "race_name", "circuit_location", "driver_name", "driver_number", "driver_nationality", "team", "grid", "race_time","points", "fastest_lap","position").withColumn("created_date", current_timestamp())
display(final_df.filter((final_df.race_year == 2020) & (final_df.race_name == "Abu Dhabi Grand Prix")).orderBy(final_df.points.desc()))


In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.race_result")